In [1]:
!npm install react-dropzone axios


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 19 packages in 2s
⠹
⠹1 package is looking for funding
⠹  run `npm fund` for details
⠹

In [2]:
!apt-get update && apt-get install -y ffmpeg


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,306 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,649 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [3]:
!pip install flask git+https://github.com/openai/whisper.git pydub

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-m6w_xx_l
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-m6w_xx_l
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━

In [4]:
!pip install pyngrok

In [5]:
from pyngrok import ngrok

In [6]:
!pip install pyngrok nest_asyncio


In [7]:
import nest_asyncio
nest_asyncio.apply()


In [8]:
from pyngrok import ngrok

# Replace the string below with your own token
ngrok.set_auth_token("2sScSNP83yzxN3ADdE61o0yKXJG_7HfXAP4EBnKu5jAmTUcM5")


In [24]:
public_url = ngrok.connect(5000)
print(" * Ngrok tunnel URL:", public_url.public_url)


 * Ngrok tunnel URL: https://0964-35-198-226-45.ngrok-free.app


In [25]:
import openai
import subprocess
import os
import math
from flask import Flask, request, render_template_string, jsonify, session
from datetime import datetime

# Configure your OpenAI client

app = Flask(__name__)
app.secret_key = os.urandom(24)  # For session management

HTML_FORM = """
<!DOCTYPE html>
<html>
  <head>
    <meta charset="UTF-8" />
    <title>Video Upload & Analysis</title>
    <style>
      body {
        background-color: #d50000;
        color: #ffffff;
        font-family: Arial, sans-serif;
        text-align: center;
        margin: 0;
        padding: 0;
      }
      h1 {
        font-size: 3em;
        margin-top: 50px;
      }
      form {
        font-size: 1.2em;
        margin: 50px auto;
        max-width: 400px;
        background-color: rgba(255, 255, 255, 0.1);
        padding: 20px;
        border-radius: 8px;
      }
      input[type="file"] {
        font-size: 1em;
        margin-bottom: 20px;
      }
      button {
        background-color: #ffffff;
        color: #d50000;
        font-size: 1.2em;
        padding: 10px 20px;
        border: none;
        border-radius: 4px;
        cursor: pointer;
      }
      button:hover {
        background-color: #ffdddd;
      }
    </style>
  </head>
  <body>
    <h1>Upload Your Video</h1>
    <form action="/upload" method="POST" enctype="multipart/form-data">
      <input type="file" name="video" accept="video/*" required />
      <br>
      <button type="submit">Analyze</button>
    </form>
  </body>
</html>
"""

def extract_audio_from_video(video_path: str, audio_path: str) -> None:
    command = [
        "ffmpeg", "-y", "-i", video_path, "-vn", "-ac", "2",
        "-ar", "44100", "-b:a", "192k", "-f", "mp3", audio_path
    ]
    process = subprocess.run(command, capture_output=True, text=True)
    if process.returncode != 0:
        raise subprocess.CalledProcessError(
            returncode=process.returncode,
            cmd=command,
            output=process.stdout,
            stderr=process.stderr
        )

def transcribe_audio_with_whisper(audio_path: str, model="whisper-1") -> str:
    with open(audio_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model=model,
            file=audio_file
        )
    return transcript.text

def generate_time_segments(script_text, interval_seconds=5, words_per_minute=150):
    words_per_second = words_per_minute / 60.0
    words_per_interval = int(words_per_second * interval_seconds)
    words = script_text.split()

    total_words = len(words)
    if words_per_interval == 0:
        return []

    num_segments = math.ceil(total_words / words_per_interval)

    segments = []
    for i in range(num_segments):
        start_idx = i * words_per_interval
        end_idx = (i + 1) * words_per_interval
        segment_words = words[start_idx:end_idx]
        segment_text = " ".join(segment_words)

        start_time = i * interval_seconds
        end_time = (i + 1) * interval_seconds

        start_time_str = f"{start_time // 60:02d}:{start_time % 60:02d}"
        end_time_str = f"{end_time // 60:02d}:{end_time % 60:02d}"
        time_stamp = f"[{start_time_str} - {end_time_str}]"

        segments.append((time_stamp, segment_text))

    return segments

def generate_overlay_prompt_for_segment(time_stamp, segment_text):
    instruction = f"""
You are a creative assistant that generates detailed image overlay prompts for video scripts.
Assume that the given text is what is spoken during a 5-second interval of a video.
Based on the following segment, generate an overlay prompt that describes the perfect image to appear during that 5-second window.
The prompt should be vivid, creative, and directly enhance the narrative.

Segment Text:
"{segment_text}"

Provide your answer in the following format:
{time_stamp}
Prompt: "Detailed description of the image to overlay..."
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": instruction}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

@app.route("/")
def index():
    return render_template_string(HTML_FORM)

@app.route("/upload", methods=["POST"])
def upload_video():
    if "video" not in request.files:
        return "No video file found!", 400

    file = request.files["video"]
    if file.filename == "":
        return "No file selected!", 400

    video_filename = "uploaded_video.mp4"
    file.save(video_filename)

    audio_filename = "extracted_audio.mp3"
    try:
        extract_audio_from_video(video_filename, audio_filename)
    except subprocess.CalledProcessError as e:
        return (
            f"<h2>Error extracting audio</h2>"
            f"<p>Command: {e.cmd}</p>"
            f"<p>Return code: {e.returncode}</p>"
            f"<p>Stdout: {e.output}</p>"
            f"<p>Stderr: {e.stderr}</p>",
            500
        )

    try:
        transcription = transcribe_audio_with_whisper(audio_filename)
    except Exception as e:
        return f"Error in transcription: {str(e)}", 500

    segments = generate_time_segments(transcription)
    results = []

    for i, (time_stamp, segment) in enumerate(segments):
        if not segment:
            continue
        prompt = generate_overlay_prompt_for_segment(time_stamp, segment)
        results.append({
            'id': i,
            'time_stamp': time_stamp,
            'segment': segment,
            'prompt': prompt
        })

    session['results'] = results

    os.remove(video_filename)
    os.remove(audio_filename)

    return render_template_string(RESULTS_TEMPLATE, results=results)

@app.route("/update_prompt", methods=["POST"])
def update_prompt():
    data = request.get_json()
    prompt_id = int(data.get('id'))
    new_prompt = data.get('prompt')

    results = session.get('results', [])
    for result in results:
        if result['id'] == prompt_id:
            result['prompt'] = new_prompt
            break

    session['results'] = results
    return jsonify({"success": True})

@app.route("/delete_prompt", methods=["POST"])
def delete_prompt():
    data = request.get_json()
    prompt_id = int(data.get('id'))

    results = session.get('results', [])
    results = [r for r in results if r['id'] != prompt_id]
    session['results'] = results

    return jsonify({"success": True})

RESULTS_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8" />
    <title>Video Analysis Results</title>
    <style>
        body {
            background-color: #d50000;
            color: #ffffff;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
        }
        .results {
            max-width: 800px;
            margin: 0 auto;
        }
        .segment {
            background: rgba(255, 255, 255, 0.1);
            padding: 15px;
            margin: 10px 0;
            border-radius: 4px;
            position: relative;
        }
        .timestamp {
            font-weight: bold;
            color: #ffd700;
            font-size: 1.1em;
        }
        .text {
            margin: 10px 0;
            line-height: 1.5em;
        }
        .prompt {
            background: rgba(255, 255, 255, 0.05);
            padding: 10px;
            border-left: 3px solid #ffd700;
            margin-top: 10px;
            position: relative;
        }
        .prompt textarea {
            width: 100%;
            min-height: 100px;
            background: rgba(255, 255, 255, 0.1);
            color: #ffffff;
            border: 1px solid rgba(255, 255, 255, 0.2);
            border-radius: 4px;
            padding: 8px;
            margin-top: 5px;
            font-family: Arial, sans-serif;
        }
        .controls {
            position: absolute;
            top: 10px;
            right: 10px;
        }
        .btn {
            background: rgba(255, 255, 255, 0.2);
            color: white;
            border: none;
            padding: 5px 10px;
            border-radius: 4px;
            cursor: pointer;
            margin-left: 5px;
        }
        .btn:hover {
            background: rgba(255, 255, 255, 0.3);
        }
        .btn-save {
            background: #4CAF50;
        }
        .btn-delete {
            background: #f44336;
        }
        h2 {
            text-align: center;
            color: #ffffff;
        }
    </style>
</head>
<body>
    <div class="results">
        <h2>Video Analysis Results</h2>
        {% for result in results %}
        <div class="segment" id="segment-{{ result.id }}">
            <div class="timestamp">{{ result.time_stamp }}</div>
            <div class="text">
                <strong>Transcript:</strong><br>
                {{ result.segment }}
            </div>
            <div class="prompt">
                <strong>Overlay Prompt:</strong>
                <div class="controls">
                    <button class="btn btn-save" onclick="savePrompt({{ result.id }})">Save</button>
                    <button class="btn btn-delete" onclick="deletePrompt({{ result.id }})">Delete</button>
                </div>
                <textarea id="prompt-{{ result.id }}">{{ result.prompt }}</textarea>
            </div>
        </div>
        {% endfor %}
    </div>

    <script>
        async function savePrompt(id) {
            const textarea = document.getElementById(`prompt-${id}`);
            const response = await fetch('/update_prompt', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({
                    id: id,
                    prompt: textarea.value
                })
            });

            if (response.ok) {
                alert('Prompt updated successfully!');
            }
        }

        async function deletePrompt(id) {
            if (!confirm('Are you sure you want to delete this prompt?')) {
                return;
            }

            const response = await fetch('/delete_prompt', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({
                    id: id
                })
            });

            if (response.ok) {
                const segment = document.getElementById(`segment-${id}`);
                segment.remove();
            }
        }
    </script>
</body>
</html>
"""

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Feb/2025 01:40:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Feb/2025 01:45:12] "POST /upload HTTP/1.1" 200 -


In [21]:
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter, AutoencoderKL
import torch
from diffusers.utils import export_to_gif, load_image, load_video, export_to_video

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [27]:
adapter_ids = {
    'guoyww_v2': "guoyww/animatediff-motion-adapter-v1-5-2", # original motion adapter
    'guoyww_v3': "guoyww/animatediff-motion-adapter-v1-5-3",
    'guoyww_sdxl': "guoyww/animatediff-motion-adapter-sdxl-beta",
    'wangfuyun': "wangfuyun/AnimateLCM",
}
model_ids = {
    'frankjoshua': "frankjoshua/toonyou_beta6",
    'SG161222': "SG161222/Realistic_Vision_V5.1_noVAE",
    'emilianJR': "emilianJR/epiCRealism",
    'stabilityai': "stabilityai/stable-diffusion-xl-base-1.0",
}
controlnet_ids = {
    'lllyasviel': "lllyasviel/control_v11f1p_sd15_depth",
    'guoyww_scr': "guoyww/animatediff-sparsectrl-scribble",
    'guoyww_rgb': "guoyww/animatediff-sparsectrl-rgb",
}
vae_ids = {
    'stabilityai': "stabilityai/sd-vae-ft-mse",
}
lora_adapter_ids = {
    'guoyww': "guoyww/animatediff-motion-lora-v1-5-3",
    'wangfuyun': "wangfuyun/AnimateLCM",
    }
depth_detector_ids = {
    'lllyasviel': "lllyasviel/Annotators",
}

In [28]:
motion_adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=dtype)
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=dtype)

model_id = "frankjoshua/toonyou_beta6" # frankjoshua/toonyou_beta6, emilianJR/epiCRealism, SG161222/Realistic_Vision_V5.1_noVAE
pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=motion_adapter, vae=vae, torch_dtype=dtype)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear", torch_dtype=dtype)

pipe.load_lora_weights(
    lora_adapter_ids['wangfuyun'], weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora",
    )
pipe.load_lora_weights(lora_adapter_ids['guoyww'], adapter_name="motion_lora") # more lora adapter at https://huggingface.co/guoyww
pipe.set_adapters("lcm-lora", [0.8])
# pipe.set_adapters(["lcm-lora", "motion_lora"], adapter_weights=[0.8, 1.0])

# Enable FreeNoise for long prompt generation
pipe.enable_free_noise(context_length=16, context_stride=4)
pipe.to(device)
# Save memory
pipe.enable_free_noise_split_inference()
pipe.unet.enable_forward_chunking()

# Can be a single prompt, or a dictionary with frame timesteps
i=100
prompt = {
    0: """
     Prompt 8:
    "A close-up of a bank teller holding a note with the words ‘Give me the money,’ her eyes rolling while a 34-year-old white man with a scruffy brown beard, messy hair, wearing a wrinkled blue uniform polo with a worn-out name tag gestures inside his jacket, trying to act intimidating but failing miserably, humorous and exaggerated expressions.""",
    i*1: """
    The 34-year-old white man with a scruffy brown beard, messy hair, wearing a wrinkled blue uniform polo with a worn-out name tag is sweating under his mask, realizing the bank heist isn't going as planned, shifting nervously while security cameras capture everything, suspenseful yet comedic tone with high contrast lighting
    """,
    i*2: """
    A 34-year-old white man with a scruffy brown beard, messy hair, wearing a wrinkled blue uniform polo with a worn-out name tag awkwardly backing away from the counter, bumping into a customer, scrambling for the door, his scarf trailing behind him in exaggerated cartoon-style motion blur.
    """
}
negative_prompt = "bad quality, worst quality, jpeg artifacts"

# Run inference
output = pipe(
    prompt=prompt,
    batch_size=1,
    negative_prompt=negative_prompt,
    num_frames=i*5,
    guidance_scale=2.5,
    num_inference_steps=10,
    generator=torch.Generator('cpu').manual_seed(42),
)

# Save video
frames = output.frames[0]
export_to_video(frames, "outputs/output1.mp4", fps=16)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--frankjoshua--toonyou_beta6/snapshots/9bb5e1c5be60ab664ec80bff429126e0844ac9fe/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--frankjoshua--toonyou_beta6/snapshots/9bb5e1c5be60ab664ec80bff429126e0844ac9fe/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (86 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['miserab

  0%|          | 0/10 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.88 GiB. GPU 0 has a total capacity of 39.56 GiB of which 4.70 GiB is free. Process 18709 has 34.85 GiB memory in use. Of the allocated memory 22.69 GiB is allocated by PyTorch, and 11.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!ffmpeg -version


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l